<a href="https://colab.research.google.com/github/Gh5al/CDMO/blob/main/SMT/SMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We tried 3 methods:
- successor approach with position variable
- boolean assignment variable + postion variable,
- int assign variable (assign a courier to each item) + position variable .<br>
The best approach is to use successor variable + position variable.<br>
For all the approaches it's fundamentel to use the lower bound constraint as it helps to reduce the search space significantly and provide a solution to instances.

In [1]:
!pip install z3-solver
from z3 import *
import numpy as np
import time
from multiprocessing import Process, Queue


In [2]:
def extract_data(filename):
  dist = []
  with open(filename,'r') as f:
    m = int(f.readline().strip())
    n = int(f.readline().strip())
    capacity = [int(s) for s in f.readline().strip().split()]
    size = [int(s) for s in f.readline().strip().split()]
    for i in range(n+1):
      dist.append([int(s) for s in f.readline().strip().split()])
  #print(m,n)
  #print(capacity)
  #print(size)
  #print(dist)
  return m,n,capacity,size,dist

In [12]:
def solve(q,solver,max_distance):
    # Try to get intermediate results
    while solver.check() == sat:
      model = solver.model()
      sub_obj = model.evaluate(max_distance).as_long()
      print(f"current_obj_value: {sub_obj}\n")
      best_obj = sub_obj
      q.put(best_obj)
      #try to improve the current obj
      solver.add(max_distance < sub_obj)

def run_with_timeout(solver,max_distance,timeout):
    q = Queue()
    p = Process(target=solve, args=(q,solver,max_distance))
    p.start()
    p.join(timeout=timeout)

    if p.is_alive():
        p.terminate()
        print("Timeout reached.")
    else:
        print("Finished before timeout.")

    # Get last model found before termination
    best_obj = 0
    while not q.empty():
        best_obj = q.get()

    if best_obj>0:
       # print("Intermediate solution found:")
        print(f"Objective:{best_obj}")
    else:
        print("No solution found in time.")



## SUCCESSOR MODEL + POSITION VARIABLE

In [23]:
def run_successor_model(filename):
  #extract data from instance file
  m,n,capacity,size,dist = extract_data(filename)
  print(f"num_couriers:{m}, num_items: {n}")
  items = list(range(n))
  locs = list(range(n + 1))
  couriers = list(range(m))
  #SUCCESSOR MODEL with position variable
  solver = Solver()
  start_time = time.time()

  # --- VARIABLES ---

  # S[k][i] = j if courier k delivers item i and then delivers item j
  S = [[Int(f"S_{k}_{i}") for i in locs] for k in couriers]

  # pos[k][i]: order index of node i visited by courier k
  pos = [[Int(f"pos_{k}_{i}") for i in items] for k in couriers]

  # distance[k]: total distance of courier k
  distance = [Int(f"distance_{k}") for k in couriers]

  #objective function
  max_distance = Int("max_distance")

  #----- CONSTRAINTS -----

  #bound the possible values the can be assigned to S variable
  for k in couriers:
    for i in locs:
      solver.add(And(S[k][i]>=0, S[k][i]<=n))

  #each item should be delivered by only one courier
  for i in items:
    count = Sum([If(S[k][i]!= i,1,0) for k in couriers])
    solver.add(count==1)

  #each courier should leave the depot
  for k in couriers:
    solver.add(S[k][n] != n)


  # ---- IMPLIED CONSTRAINTS -----

  #all courier leaves from the depot to a different place
  solver.add(Distinct([S[k][n] for k in couriers]))

  #each courier should return to depot only once ()
  for k in range(m):
    solver.add(Sum([If(S[k][i] == n,1,0) for i in range(n)])==1)

  #for each courier the successor's values should be all different
  for k in couriers:
    #a solution can be found without this constraint, but it's very slow
    solver.add(Distinct([S[k][i] for i in locs]))

  # ---- IMPLIED CONSTRAINTS -----

  #capacity constraint
  for k in couriers:
    solver.add(Sum([If(S[k][i]!= i,size[i],0) for i in items]) <= capacity[k])

  #all values in position[k] array should be different, each node appears in the route only once
  for k in couriers:
    solver.add(Distinct([pos[k][i] for i in items]))

  #prevent unconnected routes constraint
  for k in couriers:
    #num_assigned items to each courier
    num_assigned = Sum([If(S[k][i]!= i,1,0) for i in items])
    for i in items:
      #if an item is assigned to a courier k, then the position should have a value between 1 and num_assigned items to the courier k
      solver.add(Implies(S[k][i]!= i, And(pos[k][i] >= 1, pos[k][i] <= num_assigned)))
      solver.add(Implies(S[k][i]==i,pos[k][i]<0))

  #Distance calculation
  for k in couriers:
    #requires two loops because S is a symbolic variable and can't use as index in the distance matrix
      solver.add(distance[k] == Sum([Sum([If(S[k][i] == j, dist[i][j], 0) for j in locs]) for i in locs]))

  #Max distance
  for k in couriers:
      solver.add(distance[k] <= max_distance)

  lower_bound = max([dist[n][i] + dist[i][n] for i in items])
  print(f"lower_bound: {lower_bound}")
  solver.add(max_distance>=lower_bound)
  #the max distance for a courier could be to deliver n-m+1 items with longest distance between depot and delivery point
  #sorted_distances=sorted([dist[n][i]+dist[i][n] for i in range(n)],reverse=True)
  #print(sorted_distances)
  #upper_bound = sum(sorted_distances[:n-m+1])
  #print(f"upper_bound: {upper_bound}")
  #solver.add(max_distance<=upper_bound)

  #set timeout
  encoding_time = time.time() - start_time
  print(f"encoding_time: {encoding_time:3.2f} secs\n")
  # Set timeout to 5 minutes(300 secs)(include also the encoding time)
  timeout = 300 - encoding_time
  solver.set("timeout", int(timeout*1000))
  search_start_time = time.time()
  #run the searching using the Process and Queue library to abort execution after reaching timeout
  run_with_timeout(solver, max_distance, timeout)
  searching_time = time.time() - search_start_time
  final_time = searching_time + encoding_time
  print(f"Finished in: {final_time:3.2f} secs\n")


## BOOLEAN VARIABLE + POSITION VARIABLE

In [17]:
#bool var + pos var
def run_boolean_model(filename):
  #extract data from instance file
  m,n,capacity,size,dist = extract_data(filename)
  print(f"num_couriers:{m}, num_items: {n}")
  items = list(range(n))
  locs = list(range(n + 1))
  couriers = list(range(m))
  solver = Solver()
  start_time = time.time()

  # --- VARIABLES ---

  # A[k][i] = 1 if courier k delivers item i
  A = [[Bool(f"A_{k}_{i}") for i in items] for k in couriers]

  # pos[k][i]: order index of node i in the route of courier k
  pos = [[Int(f"pos_{k}_{i}") for i in items] for k in couriers]

  # distance[k]: total distance of courier k
  distance = [Int(f"distance_{k}") for k in couriers]

  #objective function
  max_distance = Int("max_distance")

  # --- CONSTRAINTS ---

  #each item should be delivered by one courier
  for i in range(n):
      solver.add(Sum([If(A[k][i],1,0) for k in couriers]) == 1)

  #Capacity constraints
  for k in couriers:
    solver.add(Sum([If(A[k][i],size[i],0) for i in items]) <= capacity[k])

  #for each courier all the position assigned should be different
  for k in couriers:
    solver.add(Distinct([pos[k][i] for i in items]))

  #prevent unconnected routes between delivered items by a courier k
  for k in couriers:
    #num_assigned items to each courier
    num_assigned = Sum([If(A[k][i],1,0) for i in items])
    for i in range(n):
      #if an item is assigned to a courier k, then the position should have a value between 1 and num_assigned items to the courier k
      solver.add(Implies(A[k][i],And(pos[k][i]>=1,pos[k][i]<=num_assigned)))
      solver.add(Implies(Not(A[k][i]),pos[k][i]<0))

  #Distance calculation
  for k in couriers:
    #num_assigned items to each courier
    num_assigned = Sum([If(A[k][i],1,0) for i in items])

    #distance from depot to the first delivery
    depot_to_first= Sum([If(And(A[k][i], pos[k][i] == 1),dist[n][i],0) for i in items])

    #create 2 for with i,j, if an item i and j are delivered, and their position only differs of 1, take all the pairs and then sum all the distances
    betweem_distance = Sum([Sum([ If(And(A[k][i],A[k][j],pos[k][j] == pos[k][i]+1),dist[i][j],0) for j in items if j!=i]) for i in items])

    #distance from last delivery to depot
    last_to_depot = Sum([If(And(A[k][i], pos[k][i] == num_assigned),dist[i][n],0) for i in items])

    #total sum
    solver.add(distance[k] == depot_to_first + betweem_distance + last_to_depot)

  #Max distance
  for k in couriers:
      solver.add(distance[k] <= max_distance)

  lower_bound = max([dist[n][i] + dist[i][n] for i in items])
  solver.add(max_distance>=lower_bound)
  print(f"lower_bound: {lower_bound}")
  #upper_bound = sum([dist[n][i] for i in range(n)]) + sum([dist[i][n] for i in range(n)])
  #print(upper_bound)
  #sorted_distances=sorted([dist[n][i]+dist[i][n] for i in range(n)],reverse=True)
  #upper_bound = sum(sorted_distances[:n-m+1])
  #print(upper_bound)
  #solver.add(max_distance<=upper_bound)
  #set timeout
  encoding_time = time.time() - start_time
  print(f"encoding_time: {encoding_time:3.2f} secs\n")
  # Set timeout to 5 minutes(300 secs)(include also the encoding time)
  timeout = 300 - encoding_time
  solver.set("timeout", int(timeout*1000))
  search_start_time = time.time()
  #run the searching using the Process and Queue library to abort execution after reaching timeout
  run_with_timeout(solver, max_distance, timeout)
  searching_time = time.time() - search_start_time
  final_time = searching_time + encoding_time
  print(f"Finished in: {final_time:3.2f} secs\n")


# MODEL 3: ASSIGN(INT) VARIABLE + POSITION VARIABLE

In [19]:
def run_int_assign_model(filename):
  #extract data from instance file
  m,n,capacity,size,dist = extract_data(filename)
  print(f"num_couriers:{m}, num_items: {n}")
  items = list(range(n))
  locs = list(range(n + 1))
  couriers = list(range(m))
  solver = Solver()
  start_time = time.time()

  # --- VARIABLES ---

  # assign[i] = k if courier k delivers item i
  assign = [Int(f"assign_{i}") for i in items]

  # pos[k][i]: order index of node i in the route of courier k
  pos = [[Int(f"pos_{k}_{i}") for i in items] for k in couriers]

  # distance[k]: total distance of courier k
  distance = [Int(f"distance_{k}") for k in couriers]

  #objective function
  max_distance = Int("max_distance")

  # --- CONSTRAINTS ---
  #each item should be delivered by a courier, bound the assign variabel
  for i in items:
    solver.add(And(assign[i] >= 0, assign[i] <= m-1))

  #capacity constraint
  for k in couriers:
    solver.add(Sum([If(assign[i]==k,size[i],0) for i in items]) <= capacity[k])

  #for each courier all the position assigned should be different
  for k in couriers:
    solver.add(Distinct([pos[k][i] for i in items]))

  #prevent unconnected routes between delivered items by a courier k
  for k in couriers:
    #num_assigned items to each courier
    num_assigned = Sum([If(assign[i]==k,1,0) for i in items])
    for i in items:
      #if an item is assigned to a courier k, then the position should have a value between 1 and num_assigned items to the courier k, other < 0
      solver.add(Implies(assign[i]==k, And(pos[k][i]>=1, pos[k][i]<=num_assigned)))
      solver.add(Implies(Not(assign[i]==k),pos[k][i]<0))

  #Distance calculation
  for k in couriers:
    #num_assigned items to each courier
    num_delivered = Sum([If(assign[i]==k,1,0) for i in items])
    #distance from depot to the first delivery
    depot_to_first= Sum([If(And(assign[i]==k,pos[k][i] == 1),dist[n][i],0) for i in items])
    #create 2 for with i,j, if an item i and j is delivered, and their position only differs of 1, take all the pairs and then sum all the distances
    betweem_distance = Sum(
        [Sum([If(And(assign[i]==k,assign[j]==k,pos[k][j] == pos[k][i]+1),dist[i][j],0) for j in items if j!=i]) for i in items])
    #distance from last delivery to depot
    last_to_depot = Sum([If(And(assign[i]==k,pos[k][i] == num_delivered) ,dist[i][n],0) for i in items])
    #total distance
    solver.add(distance[k] == depot_to_first + betweem_distance + last_to_depot)

  #Max distance
  for k in couriers:
      solver.add(distance[k] <= max_distance)

  lower_bound = max([dist[n][i] + dist[i][n] for i in items])
  solver.add(max_distance>=lower_bound)
  print(f"lower_bound: {lower_bound}")
  #upper_bound = sum([dist[n][i] for i in range(n)]) + sum([dist[i][n] for i in items])
  #print(upper_bound)
  #sorted_distances=sorted([dist[n][i]+dist[i][n] for i in items],reverse=True)
  #upper_bound = sum(sorted_distances[:n-m+1])
  #print(upper_bound)
  #solver.add(max_distance<=upper_bound)
  #set timeout
  encoding_time = time.time() - start_time
  print(f"encoding_time: {encoding_time:3.2f} secs\n")
  # Set timeout to 5 minutes(300 secs)(include also the encoding time)
  timeout = 300 - encoding_time
  solver.set("timeout", int(timeout*1000))
  search_start_time = time.time()
  #run the searching using the Process and Queue library to abort execution after reaching timeout
  run_with_timeout(solver, max_distance, timeout)
  searching_time = time.time() - search_start_time
  final_time = searching_time + encoding_time
  print(f"Finished in: {final_time:3.2f} secs\n")

# RUN MODEL

In [24]:
filename = "./inst05.dat"
run_successor_model(filename)
#run_boolean_model(filename)
#run_int_assign_model(filename)

num_couriers:2, num_items: 3
lower_bound: 160
encoding_time: 0.03 secs

current_obj_value: 252

current_obj_value: 251

current_obj_value: 250

current_obj_value: 249

current_obj_value: 248

current_obj_value: 247

current_obj_value: 246

current_obj_value: 245

current_obj_value: 244

current_obj_value: 243

current_obj_value: 242

current_obj_value: 241

current_obj_value: 240

current_obj_value: 239

current_obj_value: 238

current_obj_value: 237

current_obj_value: 236

current_obj_value: 235

current_obj_value: 234

current_obj_value: 233

current_obj_value: 232

current_obj_value: 231

current_obj_value: 230

current_obj_value: 229

current_obj_value: 228

current_obj_value: 227

current_obj_value: 226

current_obj_value: 225

current_obj_value: 224

current_obj_value: 223

current_obj_value: 222

current_obj_value: 221

current_obj_value: 220

current_obj_value: 219

current_obj_value: 218

current_obj_value: 217

current_obj_value: 216

current_obj_value: 215

current_obj_valu